# Various Searches to Explore and Describe the ETCBC Data

In [10]:
import glob, re, getpass, collections, operator, scipy.stats, sklearn.metrics
from itertools import chain
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from discreteMarkovChain import markovChain
from tf.fabric import Fabric

# format paths for Etienne or Cody
if getpass.getuser() == 'etien':
    locations = 'C:/Users/etien/Documents/github/bhsa/tf'
elif getpass.getuser() == 'cody':
    locations = '~/github/etcbc/bhsa/tf'

# load TF and BHSA data
TF = Fabric(locations=locations, modules='c', silent=True)
api = TF.load('''
              otype
              book chapter verse
              function domain txt
              typ pdp kind
              ''')
api.makeAvailableIn(globals()) # globalize TF methods

  0.00s loading features ...
   |     0.02s B otype                from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.01s B book                 from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.00s B chapter              from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.01s B verse                from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.06s B function             from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.01s B domain               from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.21s T txt                  from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.18s B typ                  from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.10s B pdp                  from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.03s B kind                 from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.00s Feature overview: 109 for nodes; 5 for edges; 1 configs; 7 computed
  4.48s All features loaded/computed - for details use loadLog()


## Discourse Particles 

In [12]:
# book lists:
lbh_books = {'1_Chronicles', '2_Chronicles', 
             'Ezra', 'Esther', 'Nehemiah'}

sbh_books = {'Genesis', 'Exodus','Leviticus', 
             'Deuteronomy','Joshua', 'Judges', 
             '1_Kings', '2_Kings', '1_Samuel',
             '2_Samuel'}

# don't load clauses with these functions
exclude_functions = {'IntS', 'Ques', 'Exst', 'ModS',
                     'NCoS', 'NCop', 'Supp', 'PrAd',
                     'Frnt', 'Intj', 'EPPr', "ExsS", 
                     "PrcS", "Voct"}

query = '''

book book={}
    clause domain=N
        phrase function=Ques
'''.format('|'.join(lbh_books|sbh_books))

S.study(query, silent=True)

S.count()

questions = list(S.fetch())

  0.00s Counting results per 100 up to 1000 ...
  0.00s Done: 5 results


In [13]:
for r in questions:
    
    clause = r[1]
    
    verse = L.u(clause, otype='verse')[0]
    
    print(T.sectionFromNode(clause))
    print(F.domain.v(clause))
    print(F.txt.v(clause))
    print(T.text(L.d(clause, otype='word')))
    print(T.text(L.d(verse, otype='word')))
    print()

('Genesis', 24, 21)
N
?N
הַֽהִצְלִ֧יחַ יְהוָ֛ה דַּרְכֹּ֖ו 
וְהָאִ֥ישׁ מִשְׁתָּאֵ֖ה לָ֑הּ מַחֲרִ֕ישׁ לָדַ֗עַת הַֽהִצְלִ֧יחַ יְהוָ֛ה דַּרְכֹּ֖ו אִם־לֹֽא׃ 

('Genesis', 8, 8)
N
N
הֲקַ֣לּוּ הַמַּ֔יִם מֵעַ֖ל פְּנֵ֥י הָֽאֲדָמָֽה׃ 
וַיְשַׁלַּ֥ח אֶת־הַיֹּונָ֖ה מֵאִתֹּ֑ו לִרְאֹות֙ הֲקַ֣לּוּ הַמַּ֔יִם מֵעַ֖ל פְּנֵ֥י הָֽאֲדָמָֽה׃ 

('Esther', 10, 2)
N
N
הֲלֹוא־הֵ֣ם כְּתוּבִ֗ים עַל־סֵ֨פֶר֙ דִּבְרֵ֣י הַיָּמִ֔ים לְמַלְכֵ֖י מָדַ֥י וּפָרָֽס׃ 
וְכָל־מַעֲשֵׂ֤ה תָקְפֹּו֙ וּגְב֣וּרָתֹ֔ו וּפָרָשַׁת֙ גְּדֻלַּ֣ת מָרְדֳּכַ֔י אֲשֶׁ֥ר גִּדְּלֹ֖ו הַמֶּ֑לֶךְ הֲלֹוא־הֵ֣ם כְּתוּבִ֗ים עַל־סֵ֨פֶר֙ דִּבְרֵ֣י הַיָּמִ֔ים לְמַלְכֵ֖י מָדַ֥י וּפָרָֽס׃ 

('Esther', 4, 5)
N
?N
מַה־זֶּ֖ה 
וַתִּקְרָא֩ אֶסְתֵּ֨ר לַהֲתָ֜ךְ מִסָּרִיסֵ֤י הַמֶּ֨לֶךְ֙ אֲשֶׁ֣ר הֶעֱמִ֣יד לְפָנֶ֔יהָ וַתְּצַוֵּ֖הוּ עַֽל־מָרְדֳּכָ֑י לָדַ֥עַת מַה־זֶּ֖ה וְעַל־מַה־זֶּֽה׃ 

('Esther', 3, 4)
N
N
הֲיַֽעַמְדוּ֙ דִּבְרֵ֣י מָרְדֳּכַ֔י 
וַיְהִ֗י כְּאָמְרָ֤ם אֵלָיו֙ יֹ֣ום וָיֹ֔ום וְלֹ֥א שָׁמַ֖ע אֲלֵיהֶ֑ם וַיַּגִּ֣ידוּ לְהָמָ֗ן לִרְאֹות֙ הֲיַֽעַמְדוּ֙ דִּבְרֵ֣י מָרְ